In [18]:
# https://stackoverflow.com/questions/33537476/mailbox-to-csv-using-python

import sys
import mailbox
import csv
import pandas as pd
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora,models
import pyLDAvis.gensim
import re
import string
import pandas as pd
import numpy as np
import mailbox
import email.utils
from email.header import decode_header
import html2text

np.random.seed(20180313)
pyLDAvis.enable_notebook()

In [19]:
def get_content(part):
    content = ''
    payload = part.get_payload()
    if isinstance(payload, str):
        content += payload
    else:
        for part in payload:
            content += get_content(part)
    return content

In [21]:
datapath = '/Users/mjschillawski/Desktop/Miscellaneous Data/'
files = ['Cornell Alum.mbox','Blog Emails.mbox','Cornell.mbox','Data.mbox','Go West, Young Man, Go West.mbox',
             'Promotions-Outdoor Retailer.mbox','Soccer.mbox','The Thread.mbox','Housing.mbox']

#files = ['Go West, Young Man, Go West.mbox']

content = []
for f in files:
    mbox = mailbox.mbox(datapath+f)
    for message in mbox:
        # https://stackoverflow.com/questions/13337528/rendered-html-to-plain-text-using-python
        try:
            thing = html2text.html2text(get_content(message))
            content.append(thing)
        except:
            pass
    print(len(content))

29580
29889
31780
33712
33838
34596
35621
39266
40719


In [22]:
def clean_text(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    
    if pd.isnull(text):
        return []
    
    try:        
        # https://stackoverflow.com/questions/21209024/python-regex-remove-all-punctuation-except-hyphen-for-unicode-string
        remove = string.punctuation
        pattern = r"[{}]".format(remove) # create the pattern
        text = re.sub(pattern,"",text)

        # break text into words
        tokenizer = RegexpTokenizer(r'\w+')
        processed_text = tokenizer.tokenize(text)
        
        # drop non-words
        processed_text = [text for text in processed_text if len(text) < 25]

        # remove stop words
        processed_text = [text.lower() for text in processed_text if text.lower() 
                          not in stopwords.words('english')]

        # remove other email words
        processed_text = [text.lower() for text in processed_text if text.lower()
                          not in ('re','fwd','r','n','t')]

        # reduce words to stems
        porter_stemmer = PorterStemmer()
        processed_text = [porter_stemmer.stem(text) for text in processed_text]

        return processed_text

    except:
        return []

In [23]:
clean_subjects = [clean_text(c) for c in content]
clean_subjects = [cs for cs in clean_subjects if len(cs) > 0]

In [ ]:
import pickle

with open("clean_subjects.pickle","wb") as file:
    pickle.dump(clean_subjects,file)

In [24]:
dictionary = corpora.Dictionary(clean_subjects)
corpus = [dictionary.doc2bow(cs) for cs in clean_subjects]

lda = models.ldamodel.LdaModel(corpus,
                              id2word=dictionary,
                              num_topics=20,
                              passes=5,
                              minimum_probability=.05)

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:995: RuntimeWarning: invalid value encountered in subtract
  result = psi(alpha) - psi(np.sum(alpha))
/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:522: RuntimeWarning: overflow encountered in multiply
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


In [25]:
lda

In [ ]:
pyLDAvis.gensim.prepare(lda,corpus,dictionary)

In [ ]:
doc_x_topic = np.array([[y for (x,y) in lda[corpus[i]]] for i in range(len(corpus))])

In [17]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42, perplexity=30)        # Instantiates Model
tsne_embedding = tsne.fit_transform(doc_x_topic)   # Fits Model/Transforms the Data
tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y']) # Creates Pandas Dataframe of TSNE results.
tsne_embedding['hue'] = doc_x_topic.argmax(axis=1) # Creates "hue" based on likeliest topic.

ValueError: setting an array element with a sequence.

In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

In [16]:
source = ColumnDataSource(
        data=dict(
            x = tsne_embedding.x,
            y = tsne_embedding.y,
            colors = [all_palettes['Category20'][20][i] for i in tsne_embedding.hue],
            topic = [i for i in tsne_embedding.hue],
            alpha = [0.7] * tsne_embedding.shape[0],
            size = [5] * tsne_embedding.shape[0]
        )
    )

hover_tsne = HoverTool(names=["train"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Topic:</span>
            <span style="font-size: 12px">@topic</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700, tools=tools_tsne, title='Newsgroups')
plot_tsne.circle('x', 'y', size='size', fill_color='colors', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="train")

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        alpha[i] = 0.7
        size[i] = 5
    }
    source.trigger('change');
""")

show(plot_tsne)

NameError: name 'ColumnDataSource' is not defined